In [ ]:
import warnings
warnings.filterwarnings("ignore", message="An issue occurred while importing 'torch-sparse'")
warnings.filterwarnings("ignore", message="An issue occurred while importing 'torch-cluster'")

import numpy as np
import networkx as nx

import torch.nn as nn
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import dropout_edge
import torch.nn as nn
import torch.nn.functional as F

from adversarial_nets import AdversarialEstimator, GraphDataset

In [ ]:
def build_peer_operator(A: np.ndarray, row_normalize: bool = True) -> np.ndarray:
    """
    Make a peer operator P from adjacency A with zero diagonal.
    Optionally row-normalize by degree; isolates pure peer effects and avoids self-loops.
    """
    P = A.copy().astype(float)
    np.fill_diagonal(P, 0.0)
    if row_normalize:
        deg = P.sum(axis=1, keepdims=True)
        with np.errstate(divide="ignore", invalid="ignore"):
            P = np.divide(P, deg, out=np.zeros_like(P), where=(deg > 0))
    return P

def simulate_linear_in_means(X: np.ndarray,
                             A: np.ndarray,
                             theta: np.ndarray,
                             noise_std: float = 0.1) -> np.ndarray:
    """
    Ground-truth simulator: y = (I - rho P)^{-1} (alpha*1 + beta*X + gamma*P X + eps)
    X: (n, 1) single scalar feature per node (for simplicity)
    A: (n, n) adjacency
    theta: [alpha, beta, gamma, rho]
    """
    alpha, beta, gamma, rho = map(float, theta)
    n = X.shape[0]
    P = build_peer_operator(A, row_normalize=True)
    I = np.eye(n)

    rhs = alpha * np.ones(n) + beta * X[:, 0] + gamma * (P @ X[:, 0]) + np.random.normal(0.0, noise_std, size=n)
    y = np.linalg.solve(I - rho * P, rhs)

    return y

def structural_model(x: np.ndarray, adjacency: np.ndarray, y0: np.ndarray, theta: np.ndarray) -> np.ndarray:
    """
    Structural mapping required by AdversarialEstimator.
    Ignores y0 here (kept for API compatibility).
    """
    return simulate_linear_in_means(x[:, [0]], adjacency, theta, noise_std=0.0)


def discriminator_factory(input_dim, hidden_dim=6, num_classes=2):
    """
    A slightly more expressive yet tiny GNN:
    - Two GCN layers (same width) with LayerNorm + ReLU
    - Edge dropout for robustness
    - Lightweight residual connection
    - Compact MLP head
    Default hidden_dim=6 keeps params roughly ~O(input_dim*6) and under ~200 for small inputs.
    """
    class RobustTinyGNN(nn.Module):
        def __init__(self, in_dim, hid_dim, num_cls):
            super().__init__()
            self.conv1 = GCNConv(in_dim, hid_dim, add_self_loops=False, normalize=True)
            self.ln1   = nn.LayerNorm(hid_dim)
            self.conv2 = GCNConv(hid_dim, hid_dim, add_self_loops=False, normalize=True)
            self.ln2   = nn.LayerNorm(hid_dim)
            self.readout = nn.Sequential(
                nn.Linear(hid_dim, 4),
                nn.ReLU(inplace=True),
                nn.Dropout(0.2),
                nn.Linear(4, num_cls)
            )

        def forward(self, data):
            x, edge_index, batch = data.x, data.edge_index, data.batch
            edge_index, _ = dropout_edge(edge_index, p=0.1, training=self.training)

            h = self.conv1(x, edge_index)
            h = self.ln1(h)
            h = F.relu(h, inplace=True)
            h = F.dropout(h, p=0.2, training=self.training)

            h2 = self.conv2(h, edge_index)
            h2 = self.ln2(h2)
            h2 = F.relu(h2, inplace=True)
            h  = h + 0.5 * h2  

            # Global readout and classification head
            h = global_mean_pool(h, batch)
            return self.readout(h)

    return RobustTinyGNN(input_dim, hidden_dim, num_classes)





def create_test_graph_dataset(n_nodes: int = 2000,
                              p_edge: float = 0.05,
                              theta_true = (0.7, 1.2, 0.8, 0.3),
                              seed: int = 42) -> GraphDataset:
    """
    Make a toy graph with one exogenous feature per node and outcomes y from the true theta.
    """
    rng = np.random.default_rng(seed)
    G = nx.erdos_renyi_graph(n_nodes, p_edge, seed=seed)
    A = nx.to_numpy_array(G, dtype=float)

    X = rng.normal(0.0, 1.0, size=(n_nodes, 1))
    P = build_peer_operator(A, row_normalize=True)

    eigvals = np.linalg.eigvals(P)
    lambda_max = float(np.max(np.abs(eigvals))) if eigvals.size else 0.0
    if lambda_max > 0 and abs(theta_true[3]) >= 0.99 / lambda_max:
        rho_safe = 0.8 / lambda_max
        theta_true = (theta_true[0], theta_true[1], theta_true[2], rho_safe)

    Y = simulate_linear_in_means(X, A, np.array(theta_true), noise_std=0.1)
    N = list(range(n_nodes))
    return GraphDataset(X=X, Y=Y, A=A, N=N)

In [ ]:
TRUE_THETA = (0.7, 1.2, 0.8, 0.3)

data = create_test_graph_dataset(n_nodes=800, p_edge=0.06, theta_true=TRUE_THETA, seed=123)
P = build_peer_operator(data.A, row_normalize=True)

eigvals = np.linalg.eigvals(P)
lam_max = float(np.max(np.abs(eigvals))) if eigvals.size else 1.0
rho_cap = (0.99 / lam_max) if lam_max > 0 else 0.99  
bounds = [(0, 5.0), (0, 5.0), (0, 5.0), (0, rho_cap)]

estimator = AdversarialEstimator(
        ground_truth_data=data,
        structural_model=structural_model,
        initial_params=[0.0, 0.0, 0.0, 0.0],
        bounds=bounds,
        discriminator_factory=discriminator_factory,
        gp_params=dict(
                initial_point_generator="sobol",
                n_initial_points=100,
                n_calls=200,
                noise=0.10,
                n_jobs=-1
            ),
        
    )

result = estimator.estimate(
    m=500,
    num_epochs=5,
    k_hops=3,
    verbose=True,
    training_params=dict(batch_size=256*2, lr=5e-2),
)
    
theta_hat = result["x"] if isinstance(result, dict) else result.x

names = ["alpha", "beta", "gamma", "rho"]
    
print("\n=== Estimated p arameters (linear-in-means, 4 params) ===")
for nm, t, th in zip(names, TRUE_THETA, theta_hat):
    print(f"{nm:>6s}: true  = {t: .4f},  est = {th: .4f},  abs.err = {abs(th - t): .4f}")